# Super-structure scenario modelling in brightway

This notebook shows how to use the bw_scenarios module, using the example of a bike.

We start by importing an SDF file (.xslx or .csv).

## Project setup

In [1]:
import bw2data as bd
import bw2io as bi

In [2]:
PROJECT_NAME = "bc24-hackathon-bike-project"
BIO_DB_NAME = "ecoinvent-3.10-biosphere"

In [3]:
bd.projects.set_current(PROJECT_NAME)

In [4]:
bd.databases

Databases dictionary with 2 object(s):
	bike_production_example
	ecoinvent-3.10-biosphere

In [5]:
xl_importer = bi.importers.ExcelImporter("bike_production_example.xlsx")

Extracted 1 worksheets in 0.02 seconds


In [6]:
# If there are unlinked exchanges because of biosphere name change,
# use the right biosphere name

In [7]:
for data_item in xl_importer.data:
    for exchange in data_item["exchanges"]:
        if exchange["type"] == "biosphere" and exchange["database"] == "biosphere3":
            print(f"Update bio db name for {exchange}")
            exchange["database"] = BIO_DB_NAME

Update bio db name for {'name': 'Carbon dioxide, fossil', 'amount': 26.6, 'database': 'biosphere3', 'unit': 'kilogram', 'categories': 'air', 'type': 'biosphere'}


In [8]:
xl_importer.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 10.34 seconds


In [9]:
xl_importer.statistics()

3 datasets
8 exchanges
0 unlinked exchanges
  


(3, 8, 0)

In [10]:
if xl_importer.statistics(print_stats=False)[2] == 0:
    xl_importer.write_database()

100%|██████████| 3/3 [00:00<?, ?it/s]

Vacuuming database 


Created database: bike_production_example


In [11]:
bike_db = bd.Database("bike_production_example")

In [12]:
for a in bike_db:
    print(f"{a} -> {a['code']}")

'natural gas production' (mega joule, NO, None) -> 6ba895871c694e9a898ade17ab6748e4
'production of bike' (unit, DK, None) -> 1d31f61bf8884a6fa5b3783270091f9d
'carbon fibre production' (kilogram, DE, None) -> 7bde0c388a1b401c95dccd0a3429bd0c


In [13]:
#make mock up for scenario.exchanges
from pprint import *
exc_dict = {}
for act in bike_db:
    if act.technosphere():
        exc_dict[act.id] = []
    
    for exc in act.technosphere():
        print("from_id:", exc.input.id, exc.input["name"], exc.amount, exc["type"])
        print("To_id:", act.id, act["name"])
        
        print("\n")
        exc_dict[act.id].append((exc.input.id, exc["type"], exc.amount))

    for exc in act.biosphere():
        print("from_id:", exc.input.id, exc.input["name"], exc.amount, exc["type"])
        print("To_id:", act.id, act["name"])
        
        print("\n")
        exc_dict[act.id].append((exc.input.id, exc["type"], exc.amount))

pprint(exc_dict)

from_id: 4363 carbon fibre production 0.01 technosphere
To_id: 4363 carbon fibre production


from_id: 4364 natural gas production 237 technosphere
To_id: 4363 carbon fibre production


from_id: 4363 carbon fibre production 2 technosphere
To_id: 4365 production of bike


from_id: 4363 carbon fibre production 0.5 technosphere
To_id: 4365 production of bike


from_id: 1034 Carbon dioxide, fossil 26.6 biosphere
To_id: 4365 production of bike


{4363: [(4363, 'technosphere', 0.01), (4364, 'technosphere', 237)],
 4365: [(4363, 'technosphere', 2),
        (4363, 'technosphere', 0.5),
        (1034, 'biosphere', 26.6)]}


In [14]:
import sys
sys.path.append("../")
from bw_scenarios.scenario import Scenario

test_scenario = Scenario("test")
test_scenario.exchanges = exc_dict


In [15]:
test_scenario.exchanges

{4363: [(4363, 'technosphere', 0.01), (4364, 'technosphere', 237)],
 4365: [(4363, 'technosphere', 2),
  (4363, 'technosphere', 0.5),
  (1034, 'biosphere', 26.6)]}

In [16]:
test_scenario.add_datapackage()

4363
[(4363, 'technosphere', 0.01), (4364, 'technosphere', 237)]
4365
[(4363, 'technosphere', 2), (4363, 'technosphere', 0.5), (1034, 'biosphere', 26.6)]


In [19]:
test_scenario.dp_static.__dict__

{'_matrix_serialize_format_type': <MatrixSerializeFormat.NUMPY: 'numpy'>,
 '_finalized': False,
 '_modified': set(),
 'fs': MemoryFS(),
 'metadata': {'profile': 'data-package',
  'name': 'fbac715d869340c19a429d7f98e183fb',
  'id': 'eb60f0f6685445a9970630995a6b7c16',
  'licenses': [{'name': 'ODC-PDDL-1.0',
    'path': 'http://opendatacommons.org/licenses/pddl/',
    'title': 'Open Data Commons Public Domain Dedication and License v1.0'}],
  'resources': [{'profile': 'data-resource',
    'format': 'npy',
    'mediatype': 'application/octet-stream',
    'name': '80529816be754105933f97c0c375274c.indices',
    'matrix': 'technosphere_matrix',
    'kind': 'indices',
    'path': '80529816be754105933f97c0c375274c.indices.npy',
    'group': '80529816be754105933f97c0c375274c',
    'category': 'vector',
    'nrows': 4},
   {'profile': 'data-resource',
    'format': 'npy',
    'mediatype': 'application/octet-stream',
    'name': '80529816be754105933f97c0c375274c.data',
    'matrix': 'technosphere_

In [25]:
import bw2calc as bc
from bw2calc.lca import prepare_lca_inputs

bike = bd.get_node(name="production of bike")
method = ('EF v3.1', 'climate change', 'global warming potential (GWP100)')

print(bike)

_, data_objs, remapping = prepare_lca_inputs(demand={bike:1}, method=method)
lca = bc.LCA(
    demand={bike: 1},
    method = method,
    data_objs=[test_scenario.dp_static],
)
lca.lci()
lca.lcia()
lca.score

'production of bike' (unit, DK, None)


OutsideTechnosphere: Can't find key 'production of bike' (unit, DK, None) in product dictionary

In [17]:

# self.fu = fu
# self.method = method

# # pre-construct the datapackages for LCA matrices
# _, data_objs, remapping = prepare_lca_inputs(demand=self.fu, method=self.method)
# self.data_objs = data_objs
# self.remapping_dicts = remapping